In [1]:
import streamlit as st
from datetime import datetime
import math

#langchain------------------------------------------------------------------------------

from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI
import streamlit as st
import os
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain
from fpdf import FPDF


os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

technology_arr=[]
levels_arr=["basic","normal","hard"]
candidate_response=""

ai_answer=""
response=[]
response_ans=[]
response_ans.append("")


In [2]:
with open("../production/rules.txt","r") as f:
    for line in f:
        key, value = line.split(': ')
        value = value.strip()  
        
        if key == "Job Description":
            job_description = value
        elif key == "Technology":
            technology_arr.append(value)
        # elif key == "Experience in Industry":
        #     experience_in_industry = value
        # elif key == "CGPA Filteration":
        #     cgpa_filteration = value
        elif key == "Instructions":
            instructions = value

In [20]:
job_description

'Looking for data scientist and machine learning'

In [3]:


# Initialize memory
memory = ConversationBufferMemory(memory_key="chat_history")

# Initialize LLM
llm = OpenAI()


In [11]:
tech=str("Technologies required :"+str(technology_arr)+" Job Descrption:"+job_description+" Other Instructions: "+instructions)


In [32]:
tech

"Technologies required :['Pytorch,Tensorflow,LLMs,LangChain'] Job Descrption:Looking for data scientist and machine learning Other Instructions: Good projects in transformers are encouraged."

In [36]:
template = """You are an expert recruiter conducting a technical interview for a software engineering position. 
Previous conversation:
{chat_history}
AI:If he answered well then increase level of questions 
AI:You ask 1 question based on the {tech_level} of questions provided. also provide very short answer. you must return level:"",question:"",answer:""
"""

prompt = PromptTemplate(
    input_variables=["chat_history", "tech_level"], template=template
)
llm_chain = LLMChain(
llm=llm,
prompt=prompt,
verbose=True,
memory=memory,
)

data=llm_chain({"tech_level":tech})
question, answer = data['text'].split("\n")[1], data['text'].split("\n")[2]
response.append(question)
response_ans.append(answer)       
# candidate_response={candidate_response}
memory.chat_memory.add_user_message(candidate_response)





> Entering new LLMChain chain...
Prompt after formatting:
You are an expert recruiter conducting a technical interview for a software engineering position. 
Previous conversation:
Human: Technologies required :['Pytorch,Tensorflow,LLMs,LangChain'] Job Descrption:Looking for data scientist and machine learning Other Instructions: Good projects in transformers are encouraged.
AI: 
Question: Can you tell me about a project you have worked on using Pytorch or Tensorflow for machine learning? 
Answer: I have recently completed a project using Pytorch for natural language processing tasks, specifically in building a language model for sentiment analysis. 
Human: ['']
Human: Technologies required :['Pytorch,Tensorflow,LLMs,LangChain'] Job Descrption:Looking for data scientist and machine learning Other Instructions: Good projects in transformers are encouraged.
AI: 
Question: Have you worked on any projects involving LLMs or LangChain for data science or machine learning?
Answer: No, I have

In [37]:
response

['Question: Can you tell me about a project you have worked on using Pytorch or Tensorflow for machine learning? ',
 'Question: Have you worked on any projects involving LLMs or LangChain for data science or machine learning?',
 'level: Intermediate']

RESUME

In [8]:
tech="Technologies required :['Pytorch,Tensorflow,LLMs,LangChain'] Job Descrption:Looking for data scientist and machine learning Other Instructions: Good projects in transformers are encouraged."

"Technologies required :['Pytorch,Tensorflow,LLMs,LangChain'] Job Descrption:Looking for data scientist and machine learning Other Instructions: Good projects in transformers are encouraged."

In [15]:
def load_pdf(file_path):
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    return documents
pdf_file_path = '../ansh_resume.pdf'
documents = load_pdf(pdf_file_path)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embeddings)

llm = OpenAI()
qa_chain = load_qa_chain(llm, chain_type="stuff")
tech=str("Technologies required :"+str(technology_arr)+" Job Descrption:"+job_description+" Other Instructions: "+instructions)

query = f"As per {tech} check whether the employee eligible for next round,If resume doesnt have required informarion return 0 , MUST RETURN OUTPUT as : Score : /100"
docs = vectorstore.similarity_search(query)
answer = qa_chain.run(input_documents=docs, question=query)


In [16]:
answer

'Score: 0/100'

In [19]:
answer.split(':')[1].split("/")[0].strip()

'0'

MARKS

In [26]:
template = """You are an expert coder conducting a technical interview for a software engineering position. 
Previous conversation:
{chat_history}
AI:You ask 1 coding question where user need to write code on paper based on the {tech_level} of questions provided. you must return question:"",answer:""
If user answered well for previous coding question(if any) then add some trick in question.
"""

prompt = PromptTemplate(
    input_variables=["chat_history", "tech_level"], template=template
)
llm_chain = LLMChain(
llm=llm,
prompt=prompt,
verbose=True,
memory=memory,
)

tech=str("Technologies required :"+str(technology_arr)+" Job Descrption:"+job_description+" Other Instructions: "+instructions)
data=llm_chain({"tech_level":tech})
question, answer = data['text'].split("\n")[1], data['text'].split("\n")[2]
response.append(question)
response_ans.append(answer)       
candidate_response=""
memory.chat_memory.add_user_message(candidate_response)





> Entering new LLMChain chain...
Prompt after formatting:
You are an expert coder conducting a technical interview for a software engineering position. 
Previous conversation:
Human: Technologies required :['Pytorch,Tensorflow,LLMs,LangChain'] Job Descrption:Looking for data scientist and machine learning Other Instructions: Good projects in transformers are encouraged.
AI: 
Question: Can you give an example of a project you have worked on using both Pytorch and Tensorflow? 
Answer: Sure, I recently worked on developing a natural language processing model using both Pytorch and Tensorflow. It involved implementing LSTMs and attention mechanisms with Pytorch, and then integrating it with a pre-trained BERT model from Tensorflow for better performance. Is that the kind of project you were looking for?
Human: ['']
Human: Technologies required :['Pytorch,Tensorflow,LLMs,LangChain'] Job Descrption:Looking for data scientist and machine learning Other Instructions: Good projects in transfo

In [23]:
response

['Question: Can you give an example of a project you have worked on using both Pytorch and Tensorflow? ']

In [27]:
question

'Question: Write a function in Python that takes in a list of sentences as input and uses Pytorch and Tensorflow to tokenize and encode them using a language model trained on LangChain. Bonus points if you can incorporate LSTMs for improved performance and return the encoded sequences in a batch format.'

CODE

In [1]:
import streamlit as st
from datetime import datetime
import math

#langchain------------------------------------------------------------------------------

from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI
import streamlit as st
import os
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain
from fpdf import FPDF


os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

technology_arr=[]
levels_arr=["basic","normal","hard"]
candidate_response=""

ai_answer=""
response=[]
response_ans=[]
response_ans.append("")


In [2]:
with open("../production/rules.txt","r") as f:
    for line in f:
        key, value = line.split(': ')
        value = value.strip()  
        
        if key == "Job Description":
            job_description = value
        elif key == "Technology":
            technology_arr.append(value)
        # elif key == "Experience in Industry":
        #     experience_in_industry = value
        # elif key == "CGPA Filteration":
        #     cgpa_filteration = value
        elif key == "Instructions":
            instructions = value

In [3]:


# Initialize memory
memory = ConversationBufferMemory(memory_key="chat_history")

# Initialize LLM
llm = OpenAI()


In [4]:
template = """You are an expert coder conducting a technical interview for a software engineering position. 
Previous conversation:
{chat_history}
AI:You ask 1 coding question where user need to write code on editor based on the DSA of questions provided. you MUST question:""
"""

prompt = PromptTemplate(
    input_variables=["chat_history", "tech_level"], template=template
)
llm_chain = LLMChain(
llm=llm,
prompt=prompt,
verbose=True,
memory=memory,
)

tech=str("Technologies required :"+str(technology_arr)+" Job Descrption:"+job_description+" Other Instructions: "+instructions)
data=llm_chain({"tech_level":tech})
question= data['text'].split("\n")[1]
response.append(question)
candidate_response=""
memory.chat_memory.add_user_message(candidate_response)



d:\abhijith\ML\pravaah\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
d:\abhijith\ML\pravaah\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
You are an expert coder conducting a technical interview for a software engineering position. 
Previous conversation:

AI:You ask 1 coding question where user need to write code on editor based on the DSA of questions provided. you MUST question:""


> Finished chain.


In [6]:
messages = memory.load_memory_variables({})


In [8]:
messages['chat_history']

"Human: Technologies required :['Pytorch,Tensorflow,LLMs,LangChain'] Job Descrption:Looking for data scientist and machine learning Other Instructions: Good projects in transformers are encouraged.\nAI: \nCan you write a function that takes in an array of integers and returns the second largest number in the array?\nHuman: "

In [8]:
question

'Interviewer: Can you write a code that uses Pytorch and Tensorflow to train a language model on a dataset using LangChain?'

ANSWER

In [ ]:
template = """You are an expert coder conducting a technical interview for a software engineering position. 
Previous conversation:
{chat_history}
AI:Check whether answer is correct code for given question {tech_level}, If YES, then add some trick in question,If NO ask technical question based on user mistake
"""

prompt = PromptTemplate(
    input_variables=["chat_history", "tech_level"], template=template
)
llm_chain = LLMChain(
llm=llm,
prompt=prompt,
verbose=True,
memory=memory,
)

tech=str("Technologies required :"+str(technology_arr)+" Job Descrption:"+job_description+" Other Instructions: "+instructions)
data=llm_chain({"tech_level":tech})
question, answer = data['text'].split("\n")[1], data['text'].split("\n")[2]
response.append(question)
response_ans.append(answer)       
candidate_response=""
memory.chat_memory.add_user_message(candidate_response)



PDF

In [9]:
def save_transcript(chat_history):
    class PDF(FPDF):
        def header(self):
            self.set_font('Arial', 'B', 12)
            self.cell(0, 10, 'Chat Transcript', 0, 1, 'C')

        def footer(self):
            self.set_y(-15)
            self.set_font('Arial', 'I', 8)
            self.cell(0, 10, 'Page ' + str(self.page_no()), 0, 0, 'C')

    pdf = PDF()
    pdf.add_page()
    pdf.set_title('Chat Transcript')
    pdf.set_author('Generated by Python Script')
    pdf.set_font('Arial', '', 12)
    for line in chat_history.split('\n'):
        pdf.cell(0, 10, line, 0, 1)
    pdf.output('chat_transcript.pdf')
    st.success("PDF created successfully!")


In [10]:
save_transcript(messages['chat_history'])

2024-05-24 12:19:49.058 
  command:

    streamlit run d:\abhijith\ML\pravaah\.venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
